#Catchall

Both tables in the raw area contain a significant number of duplicate records. We first determine which fields are duplicated and then decide how to get rid of them when creating the staging table

##Parts

In [ ]:
%%bigquery
select * from retails_raw.parts
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,p_partkey,p_type,p_size,p_brand,p_name,p_container,p_mfgr,p_retailprice,p_comment,load_time
0,316,LARGE PLATED NICKEL,1,Brand#11,orchid mint blush khaki metallic,WRAP CAN,Manufacturer#1,1216.31,ironic packages wa,2024-02-06 18:09:51.654365+00:00
1,1577,SMALL POLISHED COPPER,1,Brand#11,violet floral sienna ghost green,MED JAR,Manufacturer#1,1478.57,carefully speci,2024-02-06 18:09:51.654365+00:00
2,1676,MEDIUM POLISHED TIN,1,Brand#11,lawn black blanched slate tomato,WRAP PACK,Manufacturer#1,1577.67,slyly reg,2024-02-06 18:09:51.654365+00:00
3,2633,PROMO PLATED NICKEL,1,Brand#11,drab lemon plum rose salmon,JUMBO BAG,Manufacturer#1,1535.63,slyly exp,2024-02-06 18:09:51.654365+00:00
4,6358,ECONOMY POLISHED TIN,1,Brand#11,purple plum blush sky lavender,JUMBO BAG,Manufacturer#1,1264.35,blithely even exc,2024-02-06 18:09:51.654365+00:00


In [ ]:
%%bigquery
select (select count(*) from retails_raw.parts) as raw_parts_total_count,
  (select count(distinct p_partkey) from retails_raw.parts) as raw_meal_distinct_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_parts_total_count,raw_meal_distinct_count
0,200000,200000


In [ ]:
%%bigquery
select (select count(*) from retails_stg.Parts) as stg_parts_records,
  (select count(*) from retails_raw.parts) as raw_parts_records

Query is running:   0%|          |

Downloading:   0%|          |

,stg_parts_records,raw_parts_records
0,200000,200000


##Primary Key

###Parts

In [ ]:
%%bigquery
alter table retails_stg.Parts add primary key (p_partkey) not enforced

Executing query with job ID: 418cb4eb-48b1-4742-96d4-d99a8be8ea35
Query executing: 0.59s


ERROR:
 409 Already Exists: Constraint primary key

Location: US
Job ID: 418cb4eb-48b1-4742-96d4-d99a8be8ea35



In [ ]:
%%bigquery
select p_partkey, count(*) as duplicate_records
from retails_stg.Parts
group by p_partkey
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,p_partkey,duplicate_records


###Sales

In [ ]:
%%bigquery
alter table retails_stg.Sales add primary key (Store_ID, Dept_ID, Date) not enforced

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select Store_ID, Dept_ID, Date, count(*) as duplicate_records
from retails_stg.Sales
group by Store_ID, Dept_ID, Date
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,Store_ID,Dept_ID,Date,duplicate_records


###Partsupp

In [ ]:
%%bigquery
select ps_partkey, ps_suppkey, count(*) as duplicate_records
from retails_stg.Partsupp
group by ps_partkey, ps_suppkey
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,ps_partkey,ps_suppkey,duplicate_records


###Parts_Purchased

In [ ]:
%%bigquery
select Transaction_ID, p_partkey, count(*) as duplicate_records
from retails_stg.Parts_Purchased
group by Transaction_ID, p_partkey
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,Transaction_ID,p_partkey,duplicate_records


###Inventory

In [ ]:
%%bigquery
select Product_ID, Beginning_Inventory, Predicted_Sales, count(*) as duplicate_records
from retails_stg.Inventory
group by Product_ID, Beginning_Inventory, Predicted_Sales
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,Product_ID,Beginning_Inventory,Predicted_Sales,duplicate_records


###Supplier

In [ ]:
%%bigquery
select s_suppkey, count(*) as duplicate_records
from retails_stg.Supplier
group by s_suppkey
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,s_suppkey,duplicate_records


###Transactions

In [ ]:
%%bigquery
select Transaction_ID, count(*) as duplicate_records
from retails_stg.Transactions
group by Transaction_ID
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,Transaction_ID,duplicate_records


##Transactions

In [ ]:
%%bigquery
select * from retails_raw.transactions
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,date,store_nbr,family,sales,onpromotion,load_time
0,9,2013-01-01,1,DELI,0.0,0,2024-02-10 18:23:57.714487+00:00
1,1791,2013-01-02,1,DELI,164.069,0,2024-02-10 18:23:57.714487+00:00
2,3573,2013-01-03,1,DELI,151.582,0,2024-02-10 18:23:57.714487+00:00
3,5355,2013-01-04,1,DELI,131.411,0,2024-02-10 18:23:57.714487+00:00
4,7137,2013-01-05,1,DELI,118.613,0,2024-02-10 18:23:57.714487+00:00


In [ ]:
%%bigquery
select (select count(*) from retails_raw.transactions) as raw_transactions_total_count,
  (select count(distinct id) from retails_raw.transactions) as raw_transactions_distinct_count,
    (select count(*) - count(distinct id) from retails_raw.transactions) as duplicate_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_transactions_total_count,raw_transactions_distinct_count,duplicate_count
0,3000888,3000888,0


In [ ]:
%%bigquery
select id, count(*) as duplicate_count
from retails_raw.transactions
group by id
having count(*) > 1
order by id
limit 5

Query is running:   0%|          |

Downloading: |          |

,id,duplicate_count


In [ ]:
%%bigquery
select (select count(*) from retails_stg.Transactions) as intermediate_stg_count,
  (select count(*) from retails_raw.transactions) as raw_count

Query is running:   0%|          |

Downloading:   0%|          |

,intermediate_stg_count,raw_count
0,3000888,3000888


In [ ]:
##Clean Up

In [ ]:
%%bigquery
drop table retails_stg.snacks